In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Kaustade nimed ja klasside määramine
class_names = ["COVID19", "NORMAL", "PNEUMONIA", "TURBERCULOSIS"]
image_data = []
labels = []

# Andmete laadimine
for idx, class_name in enumerate(class_names):
    folder_path = f"Chest X_Ray Dataset/{class_name}"
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Muuda pilt 224x224 suuruseks
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Muuda värviformaat õigeks
        image_data.append(img)
        labels.append(idx)  # Määrake klassi indeks

# Muuda andmed numpy massiivideks
image_data = np.array(image_data)
labels = np.array(labels)

# Andmete jaotamine treening-, valideerimis- ja testikomplektideks
X_train, X_test, y_train, y_test = train_test_split(
    image_data, labels, test_size=0.2, random_state=42
)

# Andmete normaliseerimine
X_train = X_train / 255.0
X_test = X_test / 255.0

In [2]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)
datagen.fit(X_train)

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Kasuta eelnevalt treenitud ResNet50 mudelit
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Ära treeni eelnevalt treenitud kihti

# Täiendame mudelit
model = models.Sequential(
    [
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4, activation="softmax"),  # 4 klassi
    ]
)

model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Treenime mudelit
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=10,
    validation_data=(X_test, y_test),
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


C:\Users\raigo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.6001 - loss: 1.0849 - val_accuracy: 0.5844 - val_loss: 1.0583
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.6105 - loss: 1.0185 - val_accuracy: 0.5858 - val_loss: 1.0186
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step - accuracy: 0.6058 - loss: 0.9982 - val_accuracy: 0.5872 - val_loss: 1.0242
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.6118 - loss: 0.9867 - val_accuracy: 0.6566 - val_loss: 0.9877
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.6332 - loss: 0.9556 - val_accuracy: 0.6384 - val_loss: 0.9592
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.6159 - loss: 0.9670 - val_accuracy: 0.6265 - val_loss: 0.9547
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.6396 - loss: 0.9203 - val_accuracy: 0.6293 - val_loss: 0.9226
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.6281 - loss: 0.9295 - val_accu

In [4]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

45/45 ━━━━━━━━━━━━━━━━━━━━ 37s 769ms/step
[[  0   3 129   1]
 [  0 134 183   0]
 [  0  47 788   0]
 [  0  11 130   1]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       133
           1       0.69      0.42      0.52       317
           2       0.64      0.94      0.76       835
           3       0.50      0.01      0.01       142

    accuracy                           0.65      1427
   macro avg       0.46      0.34      0.33      1427
weighted avg       0.58      0.65      0.56      1427



C:\Users\raigo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\raigo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\raigo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

In [9]:
import matplotlib.pyplot as plt  # Kontrolli, et kasutad õiget moodulit
history = model.history
plt.plot(history.history["accuracy"], label="Treening täpsus")
plt.plot(history.history["val_accuracy"], label="Valideerimise täpsus")
plt.xlabel("Epohhid")
plt.ylabel("Täpsus")
plt.legend()
plt.show()


KeyError: 'accuracy'